In [17]:
from jsonpath_ng import jsonpath, parse

def convert_xpath_to_jsonpath(tokens):
    new_tokens = ["$"]
    for token in tokens:
        if token.type == 'ID':
            new_tokens.append(token.value)
        else:
            new_tokens.append(token_map[token.value])
    new_tokens = ''.join(new_tokens)
    return new_tokens

def evaluate_xpath_query(xpath_query, json_data):
    lexer = XpathLexer()
    xpath_tokens = list(lexer.tokenize(xpath_query))
    jsonpath_query = convert_xpath_to_jsonpath(xpath_tokens)
    jsonpath_expr = parse(jsonpath_query)
    result = [match.value for match in jsonpath_expr.find(json_data)]
    return result

def evaluate_jsonpath_query(jsonpath_query, json_data):
    jsonpath_expr = parse(jsonpath_query)
    result = [match.value for match in jsonpath_expr.find(json_data)]
    return result

In [1]:
from xpath_lexer import *
import json

with open("../tests/data/test2.json", "r") as file:
    json_data = json.load(file)

token_map = {
    "/": ".",
    "//": "..",
    "and": "&",
    "or": "|"
}

In [32]:
def convert_xpath_to_jsonpath(tokens):
    new_tokens = ['$']
    i = 0
    for token in tokens:
        if i == 0 and token.value != "/" and token.value != "//":
            new_tokens.append("..")

        if token.value == "[":
            new_tokens.append("[?")
        elif token.value in token_map:
            new_tokens.append(token_map[token.value])
        else:
            new_tokens.append(str(token.value))
        i += 1

    new_tokens = ''.join(new_tokens)
    return new_tokens

convert_xpath_to_jsonpath(list(lexer.tokenize('[year>1995 and year<1998]')))

'$..[?year>1995&year<1998]'

In [1]:
from xpath_lexer import *
path = "/library/album[count(songs/song)>=4]/title"
lexer = XpathLexer()
tokens = list(lexer.tokenize(path))

def extract_count(tokens):
    bracket_count = 1
    pos = 0
    for token in tokens:
        if token.type == '(':
            bracket_count += 1
        elif token.type == ')':
            bracket_count -= 1

        if bracket_count == 0:
            return pos
        pos += 1

def parse_count_condition(tokens):
    tokens.pop()
    tokens.pop()
    return lr_recurse(tokens, "") + ".`len`"

def lr_recurse(tokens, result):
    if not tokens:
        return result
    
    token = tokens.pop(0)
    if token.value == '/':
        result += '.'
        return lr_recurse(tokens, result)
    elif token.value == '//':
        result += '..'
        return lr_recurse(tokens, result)
    elif token.type in ['ID', 'NUMBER']:
        result += str(token.value)
        return lr_recurse(tokens, result)
    elif token.type == 'COUNT':
        tokens.pop(0)
        closing_bracket_pos = extract_count(tokens)
        count_condition_tokens = tokens[0:closing_bracket_pos]
        result += parse_count_condition(count_condition_tokens)
        return lr_recurse(tokens[closing_bracket_pos+1:], result)
    elif token.value in ['>', '<', '<=', '>=', ']']:
        result += str(token.value)
        return lr_recurse(tokens, result)
    elif token.value in ['[']:
        result += str(token.value)
        result += "?"
        return lr_recurse(tokens, result)


lr_recurse(tokens, "")

TypeError: lr_recurse() missing 1 required positional argument: 'result'

In [9]:
[1,2].pop()

AttributeError: 'int' object has no attribute 'pop'

In [7]:
extract_count(lexer.tokenize("a()b())c"))

6

In [20]:
lexer = XpathLexer()
# list(lexer.tokenize('foo[abc>1 and abc<12]'))
list(lexer.tokenize('[year>1995 and year<1998]'))

[LexToken([,'[',1,0),
 LexToken(ID,'year',1,1),
 LexToken(>,'>',1,5),
 LexToken(NUMBER,1995,1,6),
 LexToken(AND,'and',1,11),
 LexToken(ID,'year',1,15),
 LexToken(<,'<',1,19),
 LexToken(NUMBER,1998,1,20),
 LexToken(],']',1,24)]

In [ ]:
# [year>1995 and year<1998] -> [?year>1995&year<1998]

In [ ]:
evaluate_jsonpath_query('', json_data)

In [12]:
import pymongo
db_name='test'
test_client = pymongo.MongoClient('mongodb://localhost:27017/')
test_db = test_client[db_name]

def check_for_db(test_client,db_name):
    dblist = test_client.list_database_names()
    if db_name in dblist:
        print(db_name + ' ← database exists')
    else:
        print(db_name + ' ← database does NOT exists')
check_for_db(test_client,db_name) # check if database exists or not

ImportError: cannot import name '_get_object_size' from 'bson' (/opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/__init__.py)

In [97]:
from bson.json_util import dumps

collection = test_db.get_collection('library')
data = dumps(list(collection.find()))
json_data = json.loads(data)

In [11]:
!pip uninstall bson

Found existing installation: bson 0.5.10
Uninstalling bson-0.5.10:
  Would remove:
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson-0.5.10.dist-info/*
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/*
  Would not remove (might be manually added):
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/_cbson.cpython-39-darwin.so
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/_helpers.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/binary.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/code.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/codec_options.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/dbref.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/decimal128.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/errors.py
    /opt/anaconda3/envs/covidRL/lib/python3.9/site-packages/bson/int64.py
    /opt/anaconda3

In [111]:
xpath_query = "//artists/artist"
evaluate_xpath_query(xpath_query)

[{'name': 'Siti Nurhaliza', 'country': 'Malaysia'},
 {'name': 'Wham!', 'country': 'United Kingdom'},
 {'name': 'Anang Ashanty', 'country': 'Indonesia'},
 {'name': 'Anggun', 'country': 'Indonesia'},
 {'name': 'Job Bunjob Pholin', 'country': 'Thailand'},
 [{'name': 'Anang Ashanty', 'country': 'Indonesia'},
  {'name': 'Kris Dayanti', 'country': 'Indonesia'}]]